In [2]:
import torch
from datasets import IndexedDataset, WeightedDataset, load_data
from torch.utils.data import DataLoader, DistributedSampler

from utils import get_args
from architectures import load_architecture

from samplers import DistributedCustomSampler
from losses import trades_loss, apgd_loss
from tqdm.notebook import tqdm
from architectures import CustomModel, load_architecture, add_lora, set_lora_gradients #load_statedict

import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP

args = get_args()

# args.dataset = 'CIFAR10'
# args.selection_method = 'random'
# args.aug = 'aug'
# args.loss_function = 'TRADES_v2'

# args.iterations = 10
# args.pruning_ratio = 0
# args.delta = 1
# args.batch_size = 24
# args.init_lr = 0.001
# args.freeze_epochs = 5
# args.backbone = 'convnext_tiny' #deit_small_patch16_224.fb_in1k
# args.ft_type = 'full_fine_tuning'

# train_dataset, val_dataset, test_dataset, N, train_transform, transform = load_data(args) 
# # print(N)

# train_dataset = WeightedDataset(args, train_dataset, train_transform, N, prune_ratio = args.pruning_ratio,  )

# # # train_sampler = DistributedCustomSampler(args, train_dataset, num_replicas=2, rank=0, drop_last=True)
# train_sampler = DistributedSampler(train_dataset, num_replicas=2, rank=0, shuffle=True, drop_last=True)

# trainloader = DataLoader(train_dataset, batch_size=3, sampler = train_sampler, )

# model = load_architecture(args, N= 100, rank=0 )


./data


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F

# Define the modified LeNet for MNIST
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0)  # Input channels = 1 for grayscale
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)  # Average pooling
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # Adjusted for MNIST (28x28 -> 4x4 after conv and pooling)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # First convolution + activation
        x = self.pool(x)          # First pooling
        x = F.relu(self.conv2(x))  # Second convolution + activation
        x = self.pool(x)          # Second pooling
        x = x.view(-1, 16 * 4 * 4)  # Flatten the output
        x = F.relu(self.fc1(x))    # Fully connected layer + activation
        x = F.relu(self.fc2(x))    # Fully connected layer + activation
        x = self.fc3(x)           # Output layer (logits)
        return x


# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to Tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize MNIST grayscale images
])

# Datasets and Dataloaders
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Initialize model, loss function, and optimizer
device = "cuda"  # Use GPU if available
model = LeNet(num_classes=10).to(device)

from losses import apgd_loss

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Adam optimizer

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)

        # Use your custom APGD loss function
        loss_values, _ = apgd_loss(args, model, inputs, labels, optimizer)
        loss = loss_values.mean()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}")

    # Evaluate on test set
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Get predicted class
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")


Epoch [1/50], Loss: 0.5696
Test Accuracy: 66.22%
Epoch [2/50], Loss: 0.1979
Test Accuracy: 77.26%
Epoch [3/50], Loss: 0.1337
Test Accuracy: 81.48%
Epoch [4/50], Loss: 0.1012
Test Accuracy: 85.21%
Epoch [5/50], Loss: 0.0874
Test Accuracy: 79.60%
Epoch [6/50], Loss: 0.0744
Test Accuracy: 79.49%
Epoch [7/50], Loss: 0.0644
Test Accuracy: 80.46%
Epoch [8/50], Loss: 0.0584
Test Accuracy: 82.47%


KeyboardInterrupt: 

In [1]:
import torch
from datasets import IndexedDataset, WeightedDataset, load_data
from torch.utils.data import DataLoader, DistributedSampler

from utils import get_args
from architectures import load_architecture

from samplers import DistributedCustomSampler
from losses import trades_loss, apgd_loss
from tqdm.notebook import tqdm
from architectures import CustomModel, load_architecture, add_lora, set_lora_gradients #load_statedict

import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP

args = get_args()

args.dataset = 'Flowers'
args.selection_method = 'random'
args.aug = 'aug'
args.loss_function = 'APGD'

args.iterations = 10
args.pruning_ratio = 0
args.delta = 1
args.batch_size = 24
args.init_lr = 0.001
args.freeze_epochs = 5
args.backbone = 'convnext_tiny_random' #deit_small_patch16_224.fb_in1k
args.ft_type = 'full_fine_tuning'

# train_dataset, val_dataset, test_dataset, N, train_transform, transform = load_data(args) 
# # print(N)

# train_dataset = WeightedDataset(args, train_dataset, train_transform, N, prune_ratio = args.pruning_ratio,  )

# # # train_sampler = DistributedCustomSampler(args, train_dataset, num_replicas=2, rank=0, drop_last=True)
# train_sampler = DistributedSampler(train_dataset, num_replicas=2, rank=0, shuffle=True, drop_last=True)

# trainloader = DataLoader(train_dataset, batch_size=3, sampler = train_sampler, )

# Example Usage

# rank = 0
model = load_architecture(args, N= 100, rank=0 )

./data


In [9]:
def compute_gradient_norms(model):
    total_norm = 0
    for p in model.parameters():
        # print('ola',p )
        if p.grad is not None:
            print('hey')
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    total_norm = total_norm ** 0.5
    return total_norm

model.train()
compute_gradient_norms(model)

0.0